# Calibration Plot Generation Notebook

In [ ]:
import pydsm
from pydsm import postpro

import pydelmod
from pydelmod import calibplot
import panel as pn
import pandas as pd

In [ ]:
###########################################################################
# Usage:                                                                  #
# 1. make sure one of the following 3 lines is uncommented                #
# 2. change the value of timewindow (a few cells below this one)          #
###########################################################################
vartype=postpro.VarType('EC','mmhos/cm')
# vartype=postpro.VarType('FLOW','cfs')
# vartype=postpro.VarType('STAGE','ft')

# For flow plots show a 1 month period where all observed data sets have no missing values. I chose September 2011.
inst_plot_timewindow_dicts = {'FLOW': '2011-09-01:2011-09-30',
                             'EC': None, 
                             'STAGE': '2011-09-01:2011-09-30'}
location_folder = 'd:/DSM2_Calibration_Notebooks/8.2/LocationInfo/'
obs_data_folder = 'd:/DSM2_Calibration_notebooks/8.2/observedData/'
model_output_folder = 'd:/DSM2_Calibration_notebooks/8.2/modelOutput/'

## Define variable type 
(e.g. FLOW, STAGE, EC, TEMP, etc i.e. the C Part of the DSS files)

## Things that change with vartype

In [ ]:
locationfile_for_vartype={'EC': location_folder + 'calibration_ec_stations.csv',
                         'FLOW': location_folder + 'calibration_flow_stations.csv',
                         'STAGE': location_folder + 'calibration_stage_stations.csv'}
obs_dssfile_for_vartype={'EC': obs_data_folder + 'ec_merged.dss',
                       'FLOW': obs_data_folder + 'flow_merged.dss',
                       'STAGE': obs_data_folder + 'stage_merged.dss'}

## Load locations from a .csv file 
The .csv file should have atleast 'Name','BPart' and 'Description' columns

In [ ]:
locationfile=locationfile_for_vartype[vartype.name]
dfloc = postpro.load_location_file(locationfile)
locations=[postpro.Location(r['Name'],r['BPart'],r['Description']) for i,r in dfloc.iterrows()]

## Define studies
The studies are a set of a name and dss file that contains the data

In [ ]:
obs_study=postpro.Study('Observed',obs_dssfile_for_vartype[vartype.name])
# No "." allowed in study names because of issue https://github.com/holoviz/holoviews/issues/4714
study_file_map = {'DSM2v8_1_2': model_output_folder + 'historical_v81.dss',
                  'DSM2v8_2': model_output_folder + 'historical_v82b1.dss'
                  }
model_studies=[postpro.Study(name,study_file_map[name]) for name in study_file_map]
studies=[obs_study]+model_studies

## Define timewindow (optional)
Time window in the format of start_date_str - end_date_str. Specify empty string if the available data should decide the time window

In [ ]:
#timewindow='01OCT2013 - 01DEC2013'
timewindow=""
# calibration periods
hydro_calibration_time_window_str='01OCT2010 - 01OCT2012'
qual_calibration_time_window_str = '01OCT2009 - 01OCT2017'
# validation periods
hydro_validation_time_window_str='01OCT2000 - 01OCT2017'
qual_validation_time_window_str = '01OCT2000 - 01OCT2009'

# change the value of this variable
timewindow=qual_calibration_time_window_str


In [ ]:
def save_to_png(calib_plot_template,fname):
    hvobj=calib_plot_template[1][0]
    hvobj.object=hvobj.object.opts(toolbar=None) # remove the toolbar from the second row plot
    hvobj=calib_plot_template[1][0]
    hvobj.object=hvobj.object.opts(toolbar=None) # remove the toolbar from the second row plot
    calib_plot_template.save(fname)

# Build and save plot for each location

In [ ]:
def build_and_save_plot(studies, location, vartype, timewindow, write_html=False, write_png=True):
    flow_or_stage = (vartype.name == 'FLOW') or (vartype.name == 'STAGE')
    if location.name=='RSAC128-RSAC123':
        print('cross-delta flow')
        flow_or_stage = False
    flow_in_thousands = (vartype.name == 'FLOW')
        
    units=''
    # set a separate timewindow for instantaneous plots
    inst_plot_timewindow = inst_plot_timewindow_dicts[vartype.name]
    if vartype.name == 'FLOW': units='CFS'
    elif vartype.name == 'STAGE': units='FEET'
    elif vartype.name == 'EC': units='UMHOS/CM'
    calib_plot_template, metrics_df = calibplot.build_calib_plot_template(studies, location, vartype, timewindow, 
                                                            tidal_template=flow_or_stage, 
                                                            flow_in_thousands=flow_in_thousands, units=units, 
                                                            inst_plot_timewindow=inst_plot_timewindow)
    
    if write_html: calib_plot_template.save(f'{location.name}_{vartype.name}.html')
    save_to_png(calib_plot_template,f'{location.name}_{vartype.name}.png')
    return calib_plot_template, metrics_df

In [ ]:
only_these_locations = None
# if you want to create plots for a subset of the locations. Example
# only_these_locations = ['RSAC155','CHVCT000']

all_loc_metrics_df = None
for location in locations:
    if only_these_locations is None or len(only_these_locations) == 0 or location.name in only_these_locations:
        print(location)
        plot_template, metrics_df = build_and_save_plot(studies, location, vartype, timewindow, write_html=True)
        metrics_df['Location'] = [location.name, location.name]
        # move Location column to beginning
        cols = list(metrics_df)
        cols.insert(0, cols.pop(cols.index('Location')))
        metrics_df = metrics_df.loc[:, cols]

        if all_loc_metrics_df is None:
            all_loc_metrics_df = metrics_df
        else:
            all_loc_metrics_df = all_loc_metrics_df.append(metrics_df)

all_loc_metrics_df.to_csv('0_summary_statistics_'+vartype.name+'.csv')
all_loc_metrics_df.to_html('0_summary_statistics_'+vartype.name+'.html')